In [1]:
from glob import glob

In [8]:
all_negatives = [i.split('/')[-1] for i in glob('ImageFilteringData/negative_examples/*')]
black = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/black/*')]
flags = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/flags/*')]
graphs = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/graphs/*')]
icons = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/icons/*')]
maps = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/maps/*')]
sketches = [i.split('/')[-1] for i in glob('../images/neg_examples_apart/sketches/*')]

In [13]:
set(black + flags + graphs + icons + maps + sketches).difference(set(all_negatives))

set()

In [14]:
len(set(black + flags + graphs + icons + maps + sketches))

3000

In [15]:
from collections import Counter 

Counter(black + flags + graphs + icons + maps + sketches).most_common()

[('0b0d8a132668f15cffb6837f78b703344357fd45.jpg', 3),
 ('0af8890531971e4e9dd4a915a31679026eea34b1.jpg', 3),
 ('0e313a29064dad82b6dda1732dbbe492bc14cba8.jpg', 3),
 ('1eb6cc36e4b7fc0a326f874956309ab42552ea89.jpg', 3),
 ('0ac933632e9b0b23fde33e6487019b8fa4d80df4.jpg', 2),
 ('0c894893ab3dba224549961fc488183c2ca8709b.jpg', 2),
 ('0c9442917066e9d84095e8701d6028d678e3a273.jpg', 2),
 ('0a6df2209b2aa6e2945f9625093c2af69a1267b1.jpg', 2),
 ('0c05db954da4a8bed19294f466ae9cd33f1966a8.jpg', 1),
 ('00c3de98455efe3cc531f13a62e49bdeb8caf8db.jpg', 1),
 ('0c4b050e662fc6ad7e4ef96961a6e170eda79e55.jpg', 1),
 ('0c5a6586aeb3e617f2ceee4de7523c634b8da7be.jpg', 1),
 ('0c69b175d2db5dddd8f6d94dd47c724f0c0b5a2b.jpg', 1),
 ('0c6142d5af9e5c474b4c9c01331a3ceab6fb5e09.jpg', 1),
 ('0c7544c51ad37945b16218e29dc9d72bb66da107.jpg', 1),
 ('0c9803d6cb565e06f2f6912b7522225d25d39e01.jpg', 1),
 ('0a1e9c556b7be2e8767d6a3bb236eee1ffe2c3a0.jpg', 1),
 ('0a02ab218a05a5fdeb40122426b2e1f345f638b0.jpg', 1),
 ('0a79af7ed036b173131e3e71a

In [16]:
print(len(black), len(flags), len(graphs), len(icons), len(maps), len(sketches))

206 372 425 810 1042 157


In [17]:
print(len(black) + len(flags) + len(graphs) + len(icons) + len(maps) + len(sketches))

3012


In [18]:
206 + 372 + 425 + 810 + 1042 + 157

3012

In [3]:
from models.lenet import LeNet 
from models.resnet import ResNet152
from models.vgg import VGG19_BN
import torch

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [40]:
lenet = VGG19_BN()
lenet.load_state_dict(torch.load('ImageFilteringData/pretrained_cnns/vgg_test3000_28'))
lenet = lenet.to(device)
lenet.eval()

VGG19_BN(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat

In [14]:
import json
with open("ImageFilteringData/train_test_divisions/val_load_3000.json", "r") as f:
    val_load = json.loads(f.read())
    val_load = [[['ImageFilteringData/' + '/'.join(j[0].split('/')[-2:]), j[1]] for j in i] for i in val_load]

In [9]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

MODELS = {"lenet": 1, "vgg": 2, "resnet": 3}
TRANSFORMS = {1: transforms.Compose([transforms.ToTensor(), normalize]),
              2: transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(),
                transforms.ToTensor(), normalize]),
              3: transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(),
                transforms.ToTensor(), normalize])}

In [35]:
from load_data import load_image

def evaluate(net, val_load, transform, device):
    """ Evaluate the network that is trained
        Input: 
            - Network 
            - Validation data in batches
            - Desired transformation on the images 
            - Which device to run on
            - Loss function
            - Args from main 
        Out:
            - Loss on validation 
            - Correct predictions
            - Total predictions
    """
    correct = 0
    total = 0

    for batch_id, batch in enumerate(val_load):
        inputs = torch.stack(tuple([load_image(i[0], 300, 300, True, transform) for i in batch]), 0).to(device)
        targets = torch.LongTensor([i[1] for i in batch]).to(device)
        outputs = net(inputs)

        # Calculate correct predictions
        _, predicted = outputs.max(1)
        betw = predicted.eq(targets).tolist()
        for idx, val in enumerate(betw):
            if not val:
                print(batch[idx])
        #print(batch)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct, total

In [41]:
evaluate(lenet, val_load, TRANSFORMS[2], device)

['ImageFilteringData/negative_examples/0a4d74c2b41e55eec92aab38eb275a954a6dab5a.jpg', 0]
['ImageFilteringData/positive_examples/0da056f01f53abbb2c765010ddb0020ec78ecfff.jpg', 1]
['ImageFilteringData/positive_examples/008cb2e3d1d74c887306b1cb4ecfade72e94bd4a.jpg', 1]
['ImageFilteringData/negative_examples/004caa652f6ef62b84f6e2b1a94e32f95e4404aa.jpg', 0]
['ImageFilteringData/negative_examples/1e00b9440c3c077c9397c1141711ce19cc773bbc.jpg', 0]
['ImageFilteringData/negative_examples/0c723f902aed477a5b520e9bf9a660f7f51178af.jpg', 0]
['ImageFilteringData/positive_examples/0d262d627a5dba503b77cd717ecb16ef7944afd6.jpg', 1]
['ImageFilteringData/negative_examples/005db24b91e4346614d6639989b412d9d9a37772.jpg', 0]
['ImageFilteringData/negative_examples/0cccb958e319a81e40ab445003cbc67421dc004c.jpg', 0]
['ImageFilteringData/negative_examples/00ca5a544d61f400f9ebfd6659907c9eaf843d51.jpg', 0]
['ImageFilteringData/negative_examples/0006332e16c548e4ba8d38deb2e18734e622f7f1.jpg', 0]


(589, 600)